In [41]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Prueba Tecnica Cala Analytics/sentiment-analysis

Mounted at /gdrive
/gdrive/My Drive/Prueba Tecnica Cala Analytics/sentiment-analysis


In [1]:
!pip install google-play-scraper

     |████████████████████████████████| 52 kB 869 kB/s eta 0:00:011
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.5-py3-none-any.whl size=24484 sha256=f74580a9a6bbbf1cc914adaf72b918a36a5026914957db921bb4e0d31643e005
  Stored in directory: /root/.cache/pip/wheels/4a/26/18/48fda51f20c9e550c735fa6f3a6887dc8836f8d709a3cf8a9c
Successfully built google-play-scraper


In [2]:
!pip install autokeras

     |████████████████████████████████| 162 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 52.6 MB/s 
     |████████████████████████████████| 462 kB 40.6 MB/s 


In [16]:
from google_play_scraper import app
import pandas as pd
import autokeras as ak
import numpy as np

# Data acquisition

In [17]:
from google_play_scraper import Sort, reviews_all


us_reviews = reviews_all(
    'com.clarocolombia.miclaro',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)
df = pd.DataFrame(np.array(us_reviews),columns=['review'])
df = df.join(pd.DataFrame(df.pop('review').tolist()))
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHL6eLxiN3a-wG6n6ZBk_xy3UTKRfZsrrXkQXg...,Carlos Suarez Fontalvo,https://play-lh.googleusercontent.com/a-/AOh14...,Cuando inicia pide permisos de administrador,3,0,13.1.2,2022-05-16 09:56:18,"¡Buen día Carlos! Gracias por informarnos, déj...",2022-05-16 16:11:17
1,gp:AOqpTOGwrOkCjvWiuFc7vy16bN5obNX-LpL_mwjMAJI...,Samuel Alvarez Alvarez,https://play-lh.googleusercontent.com/a-/AOh14...,Ay va,4,0,13.1.2,2022-05-15 23:55:43,¡Buen día Sanuel! Gracias por tu calificación.,2022-05-16 16:03:47
2,gp:AOqpTOFgP1jpj4VTYkmgdahwE3pVhGn0t34Ujg15djM...,alberto ortiz,https://play-lh.googleusercontent.com/a-/AOh14...,Muy buena,5,0,13.1.2,2022-05-15 17:06:08,¡Muy buen día Alberto! Gracias por tu comentar...,2022-05-16 14:19:19
3,gp:AOqpTOHevOQutSRJd8INkHsUj8vto9P1NDG9Im_2zV3...,Ivan Gonzalez Muñoz,https://play-lh.googleusercontent.com/a-/AOh14...,Excelente hasta el momento el servicio Aunque ...,5,0,13.1.2,2022-05-15 11:28:34,"¡Buen día Ivan! Gracias por tu calificación, p...",2022-05-16 12:48:16
4,gp:AOqpTOES1a4a5a2im6fVwgWw_bvY08L8OxMOC_5WNqJ...,Alejandra Martin Higuera,https://play-lh.googleusercontent.com/a-/AOh14...,"Presenta muchos errores la aplicación, varios ...",1,0,13.1.2,2022-05-14 11:33:53,¡Hola Alejandra! Queremos brindarte la mejor e...,2022-05-14 16:36:03


# Preprocessing

In [18]:
def get_positive_comment(row):
    return 1 if(row>3) else 0
    
df['positiveComment'] = df['score'].apply(get_positive_comment)

df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,positiveComment
0,gp:AOqpTOHL6eLxiN3a-wG6n6ZBk_xy3UTKRfZsrrXkQXg...,Carlos Suarez Fontalvo,https://play-lh.googleusercontent.com/a-/AOh14...,Cuando inicia pide permisos de administrador,3,0,13.1.2,2022-05-16 09:56:18,"¡Buen día Carlos! Gracias por informarnos, déj...",2022-05-16 16:11:17,0
1,gp:AOqpTOGwrOkCjvWiuFc7vy16bN5obNX-LpL_mwjMAJI...,Samuel Alvarez Alvarez,https://play-lh.googleusercontent.com/a-/AOh14...,Ay va,4,0,13.1.2,2022-05-15 23:55:43,¡Buen día Sanuel! Gracias por tu calificación.,2022-05-16 16:03:47,1
2,gp:AOqpTOFgP1jpj4VTYkmgdahwE3pVhGn0t34Ujg15djM...,alberto ortiz,https://play-lh.googleusercontent.com/a-/AOh14...,Muy buena,5,0,13.1.2,2022-05-15 17:06:08,¡Muy buen día Alberto! Gracias por tu comentar...,2022-05-16 14:19:19,1
3,gp:AOqpTOHevOQutSRJd8INkHsUj8vto9P1NDG9Im_2zV3...,Ivan Gonzalez Muñoz,https://play-lh.googleusercontent.com/a-/AOh14...,Excelente hasta el momento el servicio Aunque ...,5,0,13.1.2,2022-05-15 11:28:34,"¡Buen día Ivan! Gracias por tu calificación, p...",2022-05-16 12:48:16,1
4,gp:AOqpTOES1a4a5a2im6fVwgWw_bvY08L8OxMOC_5WNqJ...,Alejandra Martin Higuera,https://play-lh.googleusercontent.com/a-/AOh14...,"Presenta muchos errores la aplicación, varios ...",1,0,13.1.2,2022-05-14 11:33:53,¡Hola Alejandra! Queremos brindarte la mejor e...,2022-05-14 16:36:03,0


# Classifier Training

In [19]:
X_train = df["content"].values
y_train = df["positiveComment"].values
clf = ak.TextClassifier(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the classifier.
clf.fit(X_train,y_train, epochs=2)

Trial 2 Complete [00h 00m 12s]
val_loss: 0.6976454257965088

Best val_loss So Far: 0.5399093627929688
Total elapsed time: 00h 00m 27s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
52/52 [==============================] - 8s 127ms/step - loss: 0.5138 - accuracy: 0.7440
Epoch 2/2
52/52 [==============================] - 6s 122ms/step - loss: 0.3822 - accuracy: 0.8634
INFO:tensorflow:Assets written to: .\text_classifier\best_model\assets


Save model

In [20]:
clf.export_model().save("models/classifier")

INFO:tensorflow:Assets written to: models/classifier\assets


# Regressor Training

In [6]:
X_train = df["content"].values
y_train = df["score"].values
rgs = ak.TextRegressor(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the classifier.
rgs.fit(X_train,y_train, epochs=2)

Trial 2 Complete [00h 00m 06s]
val_loss: 2.837764024734497

Best val_loss So Far: 2.837764024734497
Total elapsed time: 00h 00m 11s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
52/52 [==============================] - 3s 35ms/step - loss: 2.8280 - mean_squared_error: 2.8280
Epoch 2/2
52/52 [==============================] - 2s 36ms/step - loss: 2.2563 - mean_squared_error: 2.2563
INFO:tensorflow:Assets written to: .\text_regressor\best_model\assets


save model

In [7]:
rgs.export_model().save("models/regressor")

INFO:tensorflow:Assets written to: models/regressor\assets
